# 编译与运行环境

C是静态语言,需要编译后才能执行.而编译的工具就是编译器.最常见的C编译器有:

+ `GNU C`开源大佬,linux上编译器的的代名词,在osx上也可以使用homebrew安装,windows上则有MinGW作为其移植版.
+ `llvm+Clang`苹果资助的编译器,也是osx下默认的编译器.有不少黑科技,比如C++的repl,jit技术等,一点不像保守的苹果资助的东西.
+ `MSVC`微软家的御用编译器,vs内置的编译器.只能在windows下使用,要在命令行中使用需要将其加入环境变量`Path`

## GNU C

`GNU C`是标准C的一个实现,当然在满足标准C规范的前提下他还有一些其他扩展,比如支持函数内嵌套定义函数,本文使用的就是GNU C.GNU C使用的编译器叫做gcc,一般来说linux下的C编译器就是gcc,但在其他平台上也并不是没有gcc的实现:

+ mac osx上可以使用`homebrew`安装gcc,安装好后,由于osx固定的将gcc命令链接到clang.要使用的话只能使用`gcc-7`形式的命令,其中7是指gcc的大版本.

+ windows上则可以安装`mingw`来获取gnu的环境,可以通过`anaconda`安装它,需要注意的是`mingw`的安装可能会在你为python安装需要用C编译的包时带来一些错误.所以谨慎使用.

gcc工具的版本更新是很快的,一般来书并不需要追求新版本,像windows上使用`anaconda`安装的话默认版本是4.7版本,其实我们完全可以就使用这个版本的gcc.它足够稳定可靠.


## C 语言与编译

对于C来说,编译其本质就是将源文件转化为二进制库文件或者可执行文件的过程,我们以一个小程序来作为例子,使用`gnu gcc`工具来演示整个编译过程:

In [1]:
%%writefile ../codes/C1_C_tools/compiler/section1/src/helloworld.c
#include <stdio.h>

int main(){
    printf("hello world");
    return 0;
}

Overwriting ../codes/C1_C_tools/compiler/section1/src/helloworld.c


## 步骤一:解读头文件,结合源文件,组合生成真正的待编译代码
    
这一步可以使用`-E`指令实现.这个过程只激活预处理,不生成文件,而是输出到标准输出.因此你需要把它重定向到一个输出文件里.,一般来说我们用后缀`.i`保存这个文件.

这一步实际上做这些事儿:

+ 宏的替换
+ 注释的消除
+ 还有找到相关的库文件
+ 执行模板,将模板翻译成源码的实现
+ 将#include文件的全部内容插入.若用`<>`括起文件则在系统的INCLUDE目录中寻找文件,若用`" "`括起文件则在当前目录中寻找文件.

In [2]:
!gcc -E ../codes/C1_C_tools/compiler/section1/src/helloworld.c

# 1 "../codes/C1_C_tools/compiler/section1/src/helloworld.c"
# 1 "<built-in>" 1
# 1 "<built-in>" 3
# 329 "<built-in>" 3
# 1 "<command line>" 1
# 1 "<built-in>" 2
# 1 "../codes/C1_C_tools/compiler/section1/src/helloworld.c" 2
# 1 "/usr/include/stdio.h" 1 3 4
# 64 "/usr/include/stdio.h" 3 4
# 1 "/usr/include/sys/cdefs.h" 1 3 4
# 533 "/usr/include/sys/cdefs.h" 3 4
# 1 "/usr/include/sys/_symbol_aliasing.h" 1 3 4
# 534 "/usr/include/sys/cdefs.h" 2 3 4
# 599 "/usr/include/sys/cdefs.h" 3 4
# 1 "/usr/include/sys/_posix_availability.h" 1 3 4
# 600 "/usr/include/sys/cdefs.h" 2 3 4
# 65 "/usr/include/stdio.h" 2 3 4
# 1 "/usr/include/Availability.h" 1 3 4
# 172 "/usr/include/Availability.h" 3 4
# 1 "/usr/include/AvailabilityInternal.h" 1 3 4
# 173 "/usr/include/Availability.h" 2 3 4
# 66 "/usr/include/stdio.h" 2 3 4

# 1 "/usr/include/_types.h" 1 3 4
# 27 "/usr/include/_types.h" 3 4
# 1 "/usr/include/sys/_types.h" 1 3 4
# 33 "/usr/include/sys/_types.h" 3 4
# 1 "/usr/include/machine/_types.h" 1 3 4

## 步骤二:将代码转译成汇编语言

这一步可以使用`-S`指令实现.这个过程就是将上一步的这串字符转译成汇编代码.汇编代码说白了就是二进制代码中操作与资源的的替代而已.因此说C/C++和汇编其实也就一步之遥.这也是为什么很多时候嵌入式设备使用C语言编写的原因.


生成`helloworld.s`文件,`.s`文件表示是汇编文件，用编辑器打开就都是汇编指令了


In [3]:
!gcc -o ../codes/C1_C_tools/compiler/section1/src/helloworld.s \
-S ../codes/C1_C_tools/compiler/section1/src/helloworld.c

In [4]:
!cat ../codes/C1_C_tools/compiler/section1/src/helloworld.s

	.section	__TEXT,__text,regular,pure_instructions
	.macosx_version_min 10, 11
	.globl	_main
	.align	4, 0x90
_main:                                  ## @main
	.cfi_startproc
## BB#0:
	pushq	%rbp
Ltmp0:
	.cfi_def_cfa_offset 16
Ltmp1:
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
Ltmp2:
	.cfi_def_cfa_register %rbp
	subq	$16, %rsp
	leaq	L_.str(%rip), %rdi
	movl	$0, -4(%rbp)
	movb	$0, %al
	callq	_printf
	xorl	%ecx, %ecx
	movl	%eax, -8(%rbp)          ## 4-byte Spill
	movl	%ecx, %eax
	addq	$16, %rsp
	popq	%rbp
	retq
	.cfi_endproc

	.section	__TEXT,__cstring,cstring_literals
L_.str:                                 ## @.str
	.asciz	"hello world"


.subsections_via_symbols


## 步骤三:将汇编语言的代码再编译为二进制代码

这一步使用`-c`指令,将汇编代码编译位二进制代码.注意这依然不是可执行文件,因为其中使用的函数很可能并不包含在二进制文件中.

In [5]:
!gcc -o ../codes/C1_C_tools/compiler/section1/objects/helloworld.o \
-c ../codes/C1_C_tools/compiler/section1/src/helloworld.c

In [6]:
!cat ../codes/C1_C_tools/compiler/section1/objects/helloworld.o

����                         �                          �              �                    __text          __TEXT                  *             �       �            __cstring       __TEXT          *              J                             __compact_unwind__LD            8               X     �                    __eh_frame      __TEXT          X       @       x               h            $       
            -                                       _main _printf  

## 步骤四: 链接相关的二进制代码生成可执行文件

这一步就是将与文件相关的函数库引入以生成最终的可执行文件.

In [7]:
!gcc -o ../codes/C1_C_tools/compiler/section1/bin/helloworld \
../codes/C1_C_tools/compiler/section1/objects/helloworld.o

In [8]:
!../codes/C1_C_tools/compiler/section1/bin/helloworld

hello world

# 接口与实现

C语言可以接口与实现分离,也就是说同样一份接口,后面实际使用的实现形式可以是千差万别的,可以是

+ 动态/静态库
+ 单一源文件
+ 多个源文件组合

而接口和实现也不是绑定的,这样就带来了灵活性.一般来说一个模块含有一个接口文件,它的后面可能按功能等分成多个实现用的源文件,而编译好后又是一个动态/静态库.而要用这个库的时候我们再为它写带main函数的源文件.

下面是一个例子

In [9]:
%%writefile ../codes/C1_C_tools/compiler/section2/include/binary_vector.h
#ifndef BINARY_VECTOR_H
#define BINARY_VECTOR_H//一般是文件名的大写 头文件结尾写上一行

#define T BINARY_VECTOR

typedef struct {
    float x;
    float y;
} T;

extern float VEC_mod(T);
extern T VEC_add(T,T);
extern float VEC_mul(T ,T);

#undef T

#endif

Overwriting ../codes/C1_C_tools/compiler/section2/include/binary_vector.h


In [10]:
%%writefile ../codes/C1_C_tools/compiler/section2/src/unary_operator.c
#include "binary_vector.h"
#include <math.h>
#define T BINARY_VECTOR
float VEC_mod(T a){
    float result = sqrt(a.x*a.x+a.y*a.y);
    return result;
}
#undef T

Overwriting ../codes/C1_C_tools/compiler/section2/src/unary_operator.c


In [11]:
%%writefile ../codes/C1_C_tools/compiler/section2/src/binary_operator.c
#include "binary_vector.h"
#define T BINARY_VECTOR
T  VEC_add(T a,T b){
    BINARY_VECTOR result = {a.x+b.x,a.y+b.y};
    return result;
}
float VEC_mul(T a,T b){
    float result = a.x*b.x+a.y*b.y;
    return result;
}
#undef T

Overwriting ../codes/C1_C_tools/compiler/section2/src/binary_operator.c


In [12]:
%%writefile ../codes/C1_C_tools/compiler/section2/test/test.c
#include <stdio.h>
#include "binary_vector.h"


int main(void){
    BINARY_VECTOR A = {1,2},B = {3,4};
    printf("mod(A)=%f\n",VEC_mod(A));
    printf("mod(B)=%f\n",VEC_mod(B));
    printf("mul(A,B)=%f\n",VEC_mul(A,B));
    BINARY_VECTOR result = VEC_add(A,B);
    printf("add(A,B)=<%f,%f>\n",result.x,result.y);
}


Overwriting ../codes/C1_C_tools/compiler/section2/test/test.c


In [13]:
!gcc -o ../codes/C1_C_tools/compiler/section2/bin/test \
-I ../codes/C1_C_tools/compiler/section2/include \
../codes/C1_C_tools/compiler/section2/src/*.c \
../codes/C1_C_tools/compiler/section2/test/test.c

In [14]:
!../codes/C1_C_tools/compiler/section2/bin/test

mod(A)=2.236068
mod(B)=5.000000
mul(A,B)=11.000000
add(A,B)=<4.000000,6.000000>


C的特点之一就是申明与实现分离,这也决定了其开发方式和python截然相反,python一般是想到哪里写到哪里,比较适合敏捷开发,而C明显更适合先整体构思好了再动手写.因此才说python适合原型开发快速迭代,而C适合写(接口明确稳定的)业务.

# 链接库

Java当初以跨平台作为重要的卖点,号称一次编译随处运行.其实C/C++当初也是这么宣传的,只不过对于C来说跨平台指的是写好源码,头文件后可以在各个平台编译运行(系统资源接口的调用除外).但是显然这种跨平台很不彻底.

实际上还有另外一种复用的方式,就是将代码编译成链接库.再在步骤四的时候链接到最终的代码中.链接库分为静态链接库和动态链接库两种.

静态链接库与动态链接库都是共享代码的方式

链接库是由第三步的`.o`文件组合而成,各个系统平台使用不同的规范实现,具体是什么规范这边不细说.总之不同平台编译出来的链接库是不能兼容的.但其实问题也不太大,完全可以编译好后共享出去.

## 链接库的使用方法

无论静态链接库还是动态链接库,命名规则都是`libxxx+后缀`,但编译的时候指定的名字就是`xxx`.而调用方法都是用`#include`导入后就可以直接使用.

很多时候我们为了隐藏代码实现,在共享的时候可以指定好平台操作系统,之后只共享出链接库和头文件.这种方式不仅可以防止代码被人篡改或者侵权,更可以精简接口.

## 静态连接库

在linux中,而静态链接库用`.a`为后缀.windows中以`.lib`为后缀.osx中也以`.a`或`.framework`为后缀.当要使用静态链接库时,连接器会找出程序所需的函数,然后将它们拷贝到执行文件,由于这种拷贝是完整的,所以一旦连接成功,静态程序库也就不再需要了.

### 静态链接库的优点 

+ 代码装载速度快，执行速度略比动态链接库快； 

+ 只需保证在开发者的计算机中有正确的库文件,在以二进制形式发布程序时不需考虑在用户的计算机上的库文件是否存在及版本问题.

### 不足之处

+ 使用静态链接生成的可执行文件体积较大,包含相同的公共代码,造成浪费

### 构建静态链接库

我们还是以之前的二维向量作为例子,一般我们用ar(archive,归档的意思)把多个目标文件集合起来生成静态链接库文件.

## 静态连接库

在linux中,而静态链接库用`.a`为后缀.windows中以`.lib`为后缀.osx中也以`.a`或`.framework`为后缀.当要使用静态链接库时,连接器会找出程序所需的函数,然后将它们拷贝到执行文件,由于这种拷贝是完整的,所以一旦连接成功,静态程序库也就不再需要了.

### 静态链接库的优点 

+ 代码装载速度快，执行速度略比动态链接库快； 

+ 只需保证在开发者的计算机中有正确的库文件,在以二进制形式发布程序时不需考虑在用户的计算机上的库文件是否存在及版本问题.

### 不足之处

+ 使用静态链接生成的可执行文件体积较大,包含相同的公共代码,造成浪费

### 构建静态链接库

我们还是以之前的二维向量作为例子,一般我们用ar(archive,归档的意思)把多个目标文件集合起来生成静态链接库文件.

In [15]:
!gcc -c \
-o ../codes/C1_C_tools/compiler/section2/objects/binary_operator.o \
-I ../codes/C1_C_tools/compiler/section2/include \
../codes/C1_C_tools/compiler/section2/src/binary_operator.c

In [16]:
!gcc -c \
-o ../codes/C1_C_tools/compiler/section2/objects/unary_operator.o \
-I ../codes/C1_C_tools/compiler/section2/include \
../codes/C1_C_tools/compiler/section2/src/unary_operator.c

In [30]:
!ar crv ../codes/C1_C_tools/compiler/section2/lib/libvector.a \
../codes/C1_C_tools/compiler/section2/objects/*.o

a - ../codes/C1_C_tools/compiler/section2/objects/binary_operator.o
a - ../codes/C1_C_tools/compiler/section2/objects/unary_operator.o


### 使用静态链接库

要调用静态库我们依然需要头文件,我们需要一个头文件来描述这个静态库中的接口.我们可以使用之前的头文件`vector.hpp`.要使用静态链接库我们需要用`-L`来指定静态库所在的地址,再使用`-l`来指定静态库的名字.注意静态库的名字.

In [31]:
!gcc \
-o ../codes/C1_C_tools/compiler/section2/bin/test_a \
-I ../codes/C1_C_tools/compiler/section2/include \
-L ../codes/C1_C_tools/compiler/section2/lib \
-l vector \
../codes/C1_C_tools/compiler/section2/test/test.c

In [34]:
!../codes/C1_C_tools/compiler/section2/bin/test_a

mod(A)=2.236068
mod(B)=5.000000
mul(A,B)=11.000000
add(A,B)=<4.000000,6.000000>


## 动态连接库

在linux中,动态链接库通常用`.so`为后缀.在windows中,动态链接库通常用`.dll`为后缀.在osx中,动态链接库通常用`.dylib`为后缀.


某个程序在运行中要调用某个动态链接库函数的时候,操作系统首先会查看所有正在运行的程序,看在内存里是否已有此库函数的拷贝了.如果有,则让其共享那一个拷贝;只有没有才链接载入.在程序运行的时候,被调用的动态链接库函数被安置在内存的某个地方,所有调用它的程序将指向这个代码段.因此,这些代码必须使用相对地址,而不是绝对地址.在编译的时候,我们需要告诉编译器,这些对象文件是用来做动态链接库的,所以要用地址不无关代码(Position Independent Code (PIC)).

注意:Linux下进行连接的缺省操作是首先连接动态库,也就是说,如果同时存在静态和动态库,不特别指定的话,将与动态库相连接.


###  动态链接库的优点 

+ 更加节省内存并减少页面交换;

+ 动态链接文件与可执行文件独立,只要输出接口不变(即名称、参数、返回值类型和调用约定不变),更换动态链接文件不会对可执行文件造成任何影响,因而极大地提高了可维护性和可扩展性;

+ 不同编程语言编写的程序只要按照函数调用约定就可以调用同一个动态函数;

+ 适用于大规模的软件开发，使开发过程独立、耦合度小,便于不同开发者和开发组织之间进行开发和测试.

### 不足之处

+ 使用动态链接库的应用程序不是自完备的,它依赖的动态模块也要存在,如果使用载入时动态链接,程序启动时发现动态模块不存在,系统将终止程序并给出错误信息.而使用运行时动态链接,系统不会终止,但由于动态库中的导出函数不可用,程序会加载失败
+ 速度比静态链接慢.当某个模块更新后,如果新模块与旧的模块不兼容,那么那些需要该模块才能运行的软件,统统撕掉.

### 编译动态链接库

动态链接库可以直接使用g++编译.需要使用`-fPIC`指定编译成的是共享库,`-shared`指定连接方式是共享型

In [20]:
!gcc \
-o ../codes/C1_C_tools/compiler/section2/lib/libvector_b.so \
-fPIC \
-shared \
-I ../codes/C1_C_tools/compiler/section2/include \
../codes/C1_C_tools/compiler/section2/src/*

In [21]:
!gcc \
-o ../codes/C1_C_tools/compiler/section2/bin/test_b \
-I ../codes/C1_C_tools/compiler/section2/include \
-L ../codes/C1_C_tools/compiler/section2/lib \
-l vector_b \
../codes/C1_C_tools/compiler/section2/test/test.c

In [24]:
!../codes/C1_C_tools/compiler/section2/bin/test_b

mod(A)=2.236068
mod(B)=5.000000
mul(A,B)=11.000000
add(A,B)=<4.000000,6.000000>


# 使用宏实现跨平台

我们说过C/C++的跨平台是源码级别的跨平台,但实际上因为操作系统功能的实现有区别,接口也有些区别,因此很多时候我们还是得考虑跨平台问题.其实这个问题python中也有,比如到3.6版本为止windows的`ctrl+c`操作都无法直接用于协程程序的中断.

在C/C++编程时我们可以像上面定义C接口时一样用宏语句判断是否需要特殊化处理.一般情况下会出问题的也就是windows,下面是特殊化处理的宏片段:

```cpp
#ifdef _MSC_VER
    #pragma warning(disable:4996)
    #include <windows.h>
    #include <io.h>
#else
    #include <unistd.h>
    #include <sys/time.h>
    #include <pthread.h>
    #define  CRITICAL_SECTION   pthread_mutex_t
    #define  _vsnprintf         vsnprintf
#endif
```
当然了更加方便的方式是无视windows.


## 总结


### gcc的常用参数

+ -o 指定输出的文件
+ -E 预编译
+ -S 编译为汇编代码
+ -c 编译位二进制文件
+ -L 选项告诉编译器去哪里找需要的库文件
+ -l -lbvector告诉编译器要链接libbvector库
+ -I 选项告诉去哪个文件夹下找头文件
+ -i 指定使用的头文件
+ -static 链接方式禁止使用动态库。

    优点：程序运行不依赖于其他库

    缺点：文件比较大

+ -shared (-G) 此链接方式尽量使用动态库，为默认选项(共享型)

    优点：生成文件比较小

    缺点：运行时需要系统提供动态库

+ -symbolic 此链接方式将在建立共享目标文件的时候,把引用绑定到全局符号上.
+ -fpic 编译器就生成位置无关目标码.适用于共享库(shared library).通常能产生更快更小的代码,但有平台限制.编译出的代码将通过全局偏移表(Global Offset Table)中的常数地址访存，动态装载器将在程序开始执行时解析GOT表项.
+ -fPIC 编译器就输出位置无关目标码.适用于动态连接(dynamic linking),即使分支需要大范围转移.该选项会针对某些特殊机型做了特殊处理,比如适合动态链接并能避免超出GOT大小限制之类的错误.而Open64仅仅支持不会导致GOT表溢出的PIC编译.

+ ***-lstdc++*** 放在编译代码最后防止因为C++语法而报错

## ar命令常用参数


### 指令参数

+ `-d` 　删除库文件中的成员文件。
+ `-m` 　变更成员文件在库文件中的次序。
+ `-p` 　显示库文件中的成员文件内容。
+ `-q` 　将问家附加在库文件末端。
+ `-r` 　将文件插入库文件中。
+ `-t` 　显示库文件中所包含的文件。
+ `-x` 　自库文件中取出成员文件。


### 选项参数

+ `a<成员文件>` 　将文件插入库文件中指定的成员文件之后。
+ `b<成员文件>` 　将文件插入库文件中指定的成员文件之前。
+ `c` 　建立库文件。
+ `f` 　为避免过长的文件名不兼容于其他系统的ar指令指令，因此可利用此参数，截掉要放入库文件中过长的成员文件名称。
+ `i<成员文件>` 　将问家插入库文件中指定的成员文件之前。
+ `o` 　保留库文件中文件的日期。
+ `s` 　若库文件中包含了对象模式，可利用此参数建立备存文件的符号表。
+ `S` 　不产生符号表。
+ `u` 　只将日期较新文件插入库文件中。
+ `v` 　程序执行时显示详细的信息。
+ `V` 　显示版本信息。

ar用来管理一种文档。这种文档中可以包含多个其他任意类别的文件。这些被包含的文件叫做这个文档的成员。ar用来向这种文档中添加、删除、解出成员。成员的原有属性（权限、属主、日期等）不会丢失。

实际上通常只有在开发中的目标静态连接库是这种格式的，所以尽管不是，我们基本可以认为ar是用来操作这种目标链接库（.a文件）的。

### 常见操作

1. 创建库文件

    通常人们使用`ar crvu liba.a a.o`这样的命令来创建一个库并把`a.o`添加进去.`c`关键字告诉`ar`需要创建一个新库文件,如果没有指定这个标志则`ar`会创建一个文件,同时会给出一个提示信息,`u`用来告诉ar如果`a.o`比库中的同名成员要新,则用新的`a.o`替换原来的.实际上用`ar -r liba.a a.o`通常就够

2. 加入新成员

    使用`ar -r liba.a b.o`即可以将`b.o`加入到`liba.a`中.默认的加入方式为`append`，即加在库的末尾.`r`关键字可以有三个修饰符`a`, `b`和`i`

    + "a"表示after,即将新成员加在指定成员之后.例如`ar -ra a.c liba.a b.c`表示将`b.c`加入`liba.a`并放在已有成员`a.c`之后；
    + "b"表示before,即将新成员加在指定成员之前.例如`ar -rb a.c liba.a b.c`;
    + "i"表示insert,跟"b"作用相同
      
3. 列出库中已有成员

    `ar -t liba.a`即可。如果加上"v"修饰符则会一并列出成员的日期等属性。

4. 删除库中成员

    `ar -d liba.a a.c`表示从库中删除`a.c`成员.如果库中没有这个成员ar也不会给出提示.如果需要列出被删除的成员或者成员不存在的信息,就加上"v"修饰符.

5. 从库中解出成员

    `ar -x liba.a b.c`

6. 调整库中成员的顺序

    使用"m"关键字。与"r"关键字一样，它也有3个修饰符"a","b", "i"。如果要将`b.c`移动到`a.c`之前，则使用`ar -mb a.c liba.a b.c`